In [1]:
import json
import csv

import pandas as pd

# Load the main sheet
keys = json.load(open("./vote_keys.json"))
main_sheet_path = './enzh_vote.xlsx'  # Replace with your file path

all_ranks = {}
all_rank_tuples = {}
for main_sheet_name in ['A1', 'A2', 'A3']:
    # Read the main sheet into a DataFrame
    df = pd.read_excel(main_sheet_path, sheet_name=main_sheet_name)
    # Display the DataFrame
    vote1 = df['Vote 1']
    vote2 = df['Vote 2']
    vote3 = df['Vote 3']

    from collections import defaultdict
    ranks = defaultdict(list)
    rank_tuples = []
    for i,k in enumerate(keys):
        # print(k)
        # print(ranks)

        ranks[k[0]].append(vote1[i])
        ranks[k[1]].append(vote2[i])
        ranks[k[2]].append(vote3[i])

        rank_tuples.append([ranks['raw'][-1],ranks['pol'][-1],ranks['trans'][-1]])
    all_rank_tuples[main_sheet_name] = rank_tuples
    all_ranks[main_sheet_name] = ranks

In [2]:
# calcluate average rank across annotators
import numpy as np
for k in all_ranks:
    for kk in all_ranks[k]:
        all_ranks[k][kk] = np.mean(all_ranks[k][kk])
all_ranks
# average rank across annotators
for k in ['raw', 'pol', 'trans']:
    avg_ranks = []
    for anno in all_ranks:
        avg_ranks.append(all_ranks[anno][k])
    print(np.mean(avg_ranks))
all_ranks


2.3283333333333336
1.4483333333333335
2.2266666666666666


{'A1': defaultdict(list, {'pol': 1.435, 'raw': 2.34, 'trans': 2.235}),
 'A2': defaultdict(list, {'pol': 1.47, 'raw': 2.345, 'trans': 2.185}),
 'A3': defaultdict(list, {'pol': 1.44, 'raw': 2.3, 'trans': 2.26})}

In [3]:

import numpy as np
from scipy.stats import kendalltau

# Annotator rankings
A1 = np.array(all_rank_tuples[list(all_rank_tuples.keys())[0]])
A2 = np.array(all_rank_tuples[list(all_rank_tuples.keys())[1]])
A3 = np.array(all_rank_tuples[list(all_rank_tuples.keys())[2]])

# Function to calculate Kendall's Tau between pairs of annotators
def calculate_kendalls_tau(A1, A2):
    taus = []
    for i in range(A1.shape[0]):
        tau, _ = kendalltau(A1[i], A2[i])
        taus.append(tau)
    return np.mean(taus)

# Calculating pairwise Kendall's Tau
tau_A1_A2 = calculate_kendalls_tau(A1, A2)
tau_A1_A3 = calculate_kendalls_tau(A1, A3)
tau_A2_A3 = calculate_kendalls_tau(A2, A3)

# Averaging Kendall's Tau to get overall agreement
average_tau = np.mean([tau_A1_A2, tau_A1_A3, tau_A2_A3])

# Print the results
print(f"Kendall's Tau between A1 and A2: {tau_A1_A2}")
print(f"Kendall's Tau between A1 and A3: {tau_A1_A3}")
print(f"Kendall's Tau between A2 and A3: {tau_A2_A3}")
print(f"Overall average Kendall's Tau: {average_tau}")



Kendall's Tau between A1 and A2: 0.5916666666666667
Kendall's Tau between A1 and A3: 0.7416666666666666
Kendall's Tau between A2 and A3: 0.6033333333333333
Overall average Kendall's Tau: 0.6455555555555555
